# Fully-Connected Deep Neural Network with random split validation

### My updated template for a binary classification, with a confusion matrix

### Metrics given:

Accuracy, Matthews Correlation Coefficient

For each class: Recall, Precision, F1 score, Specificity, Sensitivity

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import keras
from keras import regularizers, losses
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.impute import SimpleImputer as Imputer #class
from keras.utils.vis_utils import plot_model

import keras_metrics
from keras import optimizers
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler # https://jovianlin.io/feature-scaling/
from sklearn.model_selection import KFold

In [2]:
# Getting the data
dataset = pd.read_csv('HC_PA_data.csv')
dataset.head()

,ID,Csoport,Nem,SPI.raw_0_0,MFCC.mean_0_0_[E]_1,MFCC.std_0_0_[E]_1,MFCC.range_0_0_[E]_1,HNR.mean_5_5_[E],HNR.std_5_5_[E],HNR.range_5_5_[E],...,IMF_ENTROPY_RATIO.range_0_0_[E-e:-i-2-y],IMF_ENTROPY_RATIO.mean_0_0_[O-A:-o-u],IMF_ENTROPY_RATIO.std_0_0_[O-A:-o-u],IMF_ENTROPY_RATIO.range_0_0_[O-A:-o-u],IMF_ENTROPY_RATIO.mean_0_0_[v-z-Z],IMF_ENTROPY_RATIO.std_0_0_[v-z-Z],IMF_ENTROPY_RATIO.range_0_0_[v-z-Z],IMF_ENTROPY_RATIO.mean_0_0_[b-d-g-dz-dZ-d'],IMF_ENTROPY_RATIO.std_0_0_[b-d-g-dz-dZ-d'],IMF_ENTROPY_RATIO.range_0_0_[b-d-g-dz-dZ-d']
0,PA_002no,PA,no,0.702423,226.727241,15.565275,138.304191,7.702671,1.873980,8.800351,...,2.379776,1.229107,0.297684,1.656664,1.342090,0.599020,2.269212,2.035604,0.885508,3.693024
1,PA_003no,PA,no,0.368337,188.198555,20.797340,127.246116,-3.235416,1.841550,6.213561,...,0.950906,1.054856,0.154813,0.754372,1.022489,0.159252,0.522101,1.098897,0.111197,0.488349
2,PA_004no,PA,no,0.810142,245.252498,13.106253,94.247744,9.765168,1.553110,7.726234,...,2.337904,1.175937,0.285692,1.626280,1.212787,0.271315,0.985533,1.467220,0.332705,1.475147
3,PA_005no,PA,no,1.052086,271.537454,21.658928,129.618586,11.969402,1.212987,5.478905,...,2.311154,1.010327,0.270293,1.332628,1.857653,0.562689,2.146590,1.508461,0.403692,1.557925
4,PA_006no,PA,no,0.739211,249.913724,16.760158,117.845775,2.424641,4.498472,16.214943,...,3.031446,1.277202,0.288863,1.337109,1.138326,0.284632,1.042373,1.441762,0.329920,1.256846


In [3]:
# Shuffle
np.random.seed(42) # random seed is a number (or vector) used to initialize a pseudorandom number generator
dataset = dataset.reindex(np.random.permutation(dataset.index))
dataset.reset_index(inplace=True, drop=True)

# Selecting the training attributes(X) and the label(y)
X = dataset.iloc[:, 3:52].values # X = dataset.iloc[:, np.r_[3:52]].values  lets you choose multiple coloumbs
y = dataset.iloc[:,1].values

# encode target
encode = {"HC" : 0, "PA" : 1}
decode = { 0 : "HC", 1 : "PA"}

# Change target from string to binary value
y = pd.DataFrame(y).replace(encode)
print(y)

     0
0    0
1    0
2    1
3    1
4    0
..  ..
445  1
446  0
447  0
448  0
449  1

[450 rows x 1 columns]


In [4]:
# Change y to categorical
from keras.utils import np_utils
y_categorical = np_utils.to_categorical(y)
print(y_categorical)

[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 

In [5]:
# Replace NaN values in columns with columns mean values 
imputer = Imputer(missing_values=np.nan, strategy = 'mean') # an instance of the class with these properties
imputer = imputer.fit(X)         # we have to choose the columns with missing values
X = imputer.transform(X)           # replace the X values for the columns averages

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size = 0.25, random_state = 0) 

# Feature Scaling to 0, 1 range
from sklearn.preprocessing import MinMaxScaler # Normalization x_norm = (x- min(x))/(max(x)-min(x))
sc = MinMaxScaler(feature_range=(0, 1))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
# Define the network

from keras.layers import InputLayer
#print(X.shape[0])

NUM_COLS = X.shape[1]
NUM_ROWS = X.shape[0]
#input_shape=(NUM_ROWS * NUM_COLS,)

def get_sequential_dnn():
    # create model
    classifier = Sequential(name="Sequential_DNN")  # future ANN classifier, now we initialize the different hidden layers 
    # ReLu activation function for the hidden layers
    # Sigmoid for the final layer
    
    # input layer and the first hidden layer
    classifier.add(InputLayer(input_shape=(X.shape[1])))
    classifier.add(Dense(49, activation='relu', name="first_hidden_layer", input_shape=(NUM_ROWS * NUM_COLS,)))
    
    
    # tip: number of nodes in the hidden layers = average (number of nodes in the input layer and the number of nodes in the output layer) 
                                # so output_dim =(49+1)/2 = 25
        
                                # init -> initialize the weights randomly
                                # input_dim = 49
    
    # Add a dropout layer for input layer
    # More to read about dropout here: http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf
    
    classifier.add(Dropout(0.25))
    
    #Then we simply add the input-, hidden- and output-layers. 
    # Between them, we are using dropout to prevent overfitting. 
    # Note that you should always use a dropout rate between 20% and 50%. 
    # At every layer, we use “Dense” which means that the units are fully connected. 
    
    # second hidden layer
    #classifier.add(Dense(output_dim = 25, kernel_regularizer = regularizers.l2(0.01), activity_regularizer = regularizers.l1(0.01), init ='uniform', activation = 'relu', input_dim = 49))         
    classifier.add(Dense(25, activation = 'relu', name="second_hidden_layer"))
    classifier.add(Dropout(0.25))
    #classifier.add(Dropout(0.5))
        
    # third hidden layer
    #classifier.add(Dense(25, activation = 'relu', name="third_hidden_layer"))
    #classifier.add(Dropout(0.25))
    #classifier.add(Dropout(0.5))
    
    # 4th hidden layer
    #classifier.add(Dense(25, activation = 'relu', name="forth_hidden_layer"))
    #classifier.add(Dropout(0.25))
    #classifier.add(Dropout(0.5))
    
    #classifier.add(Dense(output_dim = 2, activation = 'softmax', name="output_layer"))
    classifier.add(Dense(2, activation='softmax', name="final_layer"))
    return classifier

classifier=get_sequential_dnn()
classifier.summary()
#print(len(classifier.layers))

Model: "Sequential_DNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_hidden_layer (Dense)   (None, 49)                2450      
_________________________________________________________________
dropout (Dropout)            (None, 49)                0         
_________________________________________________________________
second_hidden_layer (Dense)  (None, 25)                1250      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
final_layer (Dense)          (None, 2)                 52        
Total params: 3,752
Trainable params: 3,752
Non-trainable params: 0
_________________________________________________________________


In [7]:
BATCH_SIZE = 128 #  128
EPOCHS = 100

classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras_metrics.precision(), keras_metrics.recall(), 'acc'])
        
    # Fit the model    
    #classifier.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=150, verbose=0) 

classifier.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data = (X_test, y_test))


Epoch 1/100
Instructions for updating:
`inputs` is now automatically inferred
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
3/3 [==============================] - 0s 108ms/step - loss: 0.7194 - precision: 0.2849 - recall: 0.0565 - acc: 0.5460 - val_loss: 0.7028 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_acc: 0.5487
Epoch 2/100
3/3 [==============================] - 0s 10ms/step - loss: 0.7114 - precision: 0.3289 - recall: 0.1050 - acc: 0.5371 - val_loss: 0.6880 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_acc: 0.5487
Epoch 3/100
3/3 [==============================] - 0s 16ms/step - loss: 0.6893 - precision: 0.4229 - recall: 0.1786 - acc: 0.5549 - val_loss: 0.6791 - val_precision: 0.5000 - val_recall: 0.0196 - val_acc: 0.5487
Epoch 4/100
3/3 [==============================] - 0s 17ms/step - loss: 0.6733 - precision: 0.5408 - recall: 0.2563 - acc: 0.5964 - val_loss: 0.6729 - val_precision

3/3 [==============================] - 0s 11ms/step - loss: 0.5057 - precision: 0.7247 - recall: 0.5652 - acc: 0.7389 - val_loss: 0.4608 - val_precision: 0.8571 - val_recall: 0.7059 - val_acc: 0.8142
Epoch 37/100
3/3 [==============================] - 0s 9ms/step - loss: 0.5295 - precision: 0.6967 - recall: 0.6528 - acc: 0.7389 - val_loss: 0.4531 - val_precision: 0.8444 - val_recall: 0.7451 - val_acc: 0.8230
Epoch 38/100
3/3 [==============================] - 0s 10ms/step - loss: 0.5199 - precision: 0.7438 - recall: 0.6940 - acc: 0.7656 - val_loss: 0.4491 - val_precision: 0.8571 - val_recall: 0.7059 - val_acc: 0.8142
Epoch 39/100
3/3 [==============================] - 0s 14ms/step - loss: 0.5118 - precision: 0.7587 - recall: 0.6575 - acc: 0.7507 - val_loss: 0.4459 - val_precision: 0.9211 - val_recall: 0.6863 - val_acc: 0.8319
Epoch 40/100
3/3 [==============================] - 0s 17ms/step - loss: 0.5449 - precision: 0.6900 - recall: 0.6321 - acc: 0.7359 - val_loss: 0.4403 - val_precis

3/3 [==============================] - 0s 13ms/step - loss: 0.4510 - precision: 0.7671 - recall: 0.6727 - acc: 0.7804 - val_loss: 0.3869 - val_precision: 0.9000 - val_recall: 0.7059 - val_acc: 0.8319
Epoch 75/100
3/3 [==============================] - 0s 10ms/step - loss: 0.4602 - precision: 0.8536 - recall: 0.7216 - acc: 0.7923 - val_loss: 0.3841 - val_precision: 0.8780 - val_recall: 0.7059 - val_acc: 0.8230
Epoch 76/100
3/3 [==============================] - 0s 9ms/step - loss: 0.4758 - precision: 0.7263 - recall: 0.6433 - acc: 0.7656 - val_loss: 0.3758 - val_precision: 0.8696 - val_recall: 0.7843 - val_acc: 0.8496
Epoch 77/100
3/3 [==============================] - 0s 10ms/step - loss: 0.4588 - precision: 0.7634 - recall: 0.7213 - acc: 0.7745 - val_loss: 0.3758 - val_precision: 0.8696 - val_recall: 0.7843 - val_acc: 0.8496
Epoch 78/100
3/3 [==============================] - 0s 12ms/step - loss: 0.4464 - precision: 0.7636 - recall: 0.7507 - acc: 0.7804 - val_loss: 0.3854 - val_precis

In [8]:
score = classifier.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.35819998383522034
Test accuracy: 0.7876259088516235


In [9]:
# Predicting test labels    

pred = classifier.predict(X_test)
pred = pred.round().astype(int)
pred = np.argmax(pred, axis=1)
#pred = pred.reshape(y_test.shape)

print("Predicted:", pred)
print("Test", y_test[0])
print("Test", y_test[0].argmax(0))

# one-hot to simple test format
y_test = np.argmax(y_test, axis=1)
print("Test", y_test)


Predicted: [1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1
 1 1 1 1 0 1 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 0 1 1 1 0
 1 0 0 1 1 0 0 0 1 1 0 0 0 1 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 0 0 1 1
 0 0]
Test [0. 1.]
Test 1
Test [1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 1 0 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 0 1 1 1 0 1 1 1 0
 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1
 0 0]


In [10]:
pred = pred.astype('str')
y_test = y_test.astype('str')
pred[pred == '0'] = decode[0]
pred[pred == '1'] = decode[1]
y_test[y_test == '0'] = decode[0]
y_test[y_test == '1'] = decode[1]

print("Predicted:", pred)
print("Test", y_test)


Predicted: ['PA' 'HC' 'PA' 'PA' 'PA' 'HC' 'PA' 'PA' 'PA' 'HC' 'PA' 'HC' 'HC' 'PA'
 'PA' 'PA' 'HC' 'HC' 'PA' 'PA' 'HC' 'HC' 'PA' 'PA' 'PA' 'HC' 'PA' 'PA'
 'PA' 'PA' 'PA' 'HC' 'PA' 'HC' 'PA' 'PA' 'PA' 'PA' 'PA' 'PA' 'PA' 'HC'
 'PA' 'HC' 'PA' 'HC' 'HC' 'HC' 'PA' 'PA' 'PA' 'PA' 'HC' 'HC' 'PA' 'PA'
 'PA' 'PA' 'PA' 'HC' 'HC' 'PA' 'HC' 'HC' 'PA' 'HC' 'PA' 'PA' 'PA' 'HC'
 'PA' 'PA' 'PA' 'HC' 'PA' 'HC' 'HC' 'PA' 'PA' 'HC' 'HC' 'HC' 'PA' 'PA'
 'HC' 'HC' 'HC' 'PA' 'HC' 'PA' 'PA' 'PA' 'HC' 'PA' 'HC' 'PA' 'PA' 'HC'
 'PA' 'PA' 'PA' 'PA' 'PA' 'HC' 'HC' 'PA' 'HC' 'HC' 'HC' 'PA' 'PA' 'HC'
 'HC']
Test ['PA' 'HC' 'PA' 'PA' 'PA' 'PA' 'PA' 'HC' 'PA' 'PA' 'PA' 'PA' 'HC' 'HC'
 'HC' 'PA' 'HC' 'HC' 'PA' 'PA' 'HC' 'HC' 'PA' 'HC' 'PA' 'HC' 'PA' 'PA'
 'PA' 'PA' 'PA' 'PA' 'PA' 'HC' 'PA' 'PA' 'PA' 'PA' 'PA' 'PA' 'PA' 'HC'
 'PA' 'HC' 'PA' 'HC' 'HC' 'PA' 'PA' 'PA' 'PA' 'HC' 'HC' 'HC' 'PA' 'HC'
 'PA' 'PA' 'PA' 'HC' 'HC' 'PA' 'HC' 'HC' 'HC' 'HC' 'PA' 'PA' 'PA' 'HC'
 'PA' 'PA' 'PA' 'HC' 'PA' 'HC' 'HC' 'PA' 'HC' 'HC' 'HC

In [11]:
y_actu = pd.Series(y_test, name='True')
y_pred = pd.Series(pred, name='Pred')
df_confusion = pd.crosstab(y_pred, y_actu, colnames=['True'], rownames=['Predicted'], margins=True)
print("")
print(df_confusion)


True       HC  PA  All
Predicted             
HC         39   6   45
PA         12  56   68
All        51  62  113


In [12]:
cm_final = df_confusion.iloc[0:-1].values

cm_final = cm_final[:,[0,1]]
print(cm_final)

[[39  6]
 [12 56]]


In [13]:
#########################################################
# Statistical measures calculated from Confusion Matrix #
#########################################################
import math

# (tp + tn) / (tp + fp + tn + fn)
def get_accuracy(mx):
    [tp, fp], [fn, tn] = mx
    #print([tp, fp], [fn, tn])
    return (tp + tn) / (tp + fp + tn + fn)

# sensitivity, recall, hit rate, or true positive rate (TPR)
# tp / (tp + fn)
def get_recall(mx):
    [tp, fp], [fn, tn] = mx
    return tp / (tp + fn)

# precision or positive predictive value (PPV)
# tp / (tp + fp)
def get_precision(mx):
    [tp, fp], [fn, tn] = mx
    return tp / (tp + fp)

# harmonic mean of precision and sensitivity
# 2*((precision*recall)/(precision+recall))
def get_f1score(mx):
    return 2*((get_precision(mx)*get_recall(mx))/(get_precision(mx)+get_recall(mx)))

# specificity, selectivity or true negative rate (TNR)
# 
def get_specificity(mx):
    [tp, fp], [fn, tn] = mx
    return tp / (tp + fn)

def get_sensitivity(mx):
    [tp, fp], [fn, tn] = mx
    return tn/(tn+fp)

def get_MCC(mx):
    # Matthews Correlation Coefficient (MCC)
    [tp, fp], [fn, tn] = mx
    
    return (tp*tn-fp*fn)/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))

In [14]:
# Calculating statistical measures from Confusion Matrix
mx = cm_final
print("__________________________________")
print("")
print('Accuracy: %f' % get_accuracy(mx))
print('Matthews Correlation Coefficient: %f' % get_MCC(mx))
print("")

print("For class {}: ".format(df_confusion.columns[0]) )
print("")
print('Recall: %f' % get_recall(mx))
print('Precision: %f' % get_precision(mx))
print('F1 score: %f' % get_f1score(mx))
print('Specificity: %f' % get_specificity(mx))
print('Sensitivity: %f' % get_sensitivity(mx))

[tn, fp], [fn, tp] = mx
mx_ = [tp, fn],[fp, tn]
print("")
print("For class {}: ".format(df_confusion.columns[1]) )
print("")
print('Recall: %f' % get_recall(mx_))
print('Precision: %f' % get_precision(mx_))
print('F1 score: %f' % get_f1score(mx_))
print('Specificity: %f' % get_specificity(mx_))
print('Sensitivity: %f' % get_sensitivity(mx_))

__________________________________

Accuracy: 0.840708
Matthews Correlation Coefficient: 0.678973

For class HC: 

Recall: 0.764706
Precision: 0.866667
F1 score: 0.812500
Specificity: 0.764706
Sensitivity: 0.903226

For class PA: 

Recall: 0.903226
Precision: 0.823529
F1 score: 0.861538
Specificity: 0.903226
Sensitivity: 0.764706
